In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import networkx as nx
import matplotlib.pyplot as plt
import functions as fn
from networkx.algorithms import community
import matplotlib.patches as mpatches
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('data/ccm_manipulated_347418.csv', header=None)

In [17]:
step_amounts = list(range(5,55,5))
dfs = []
node_numbers = []
bucket_amounts = []
for i in step_amounts:
    feature = 8
    df = pd.DataFrame()
    df['sequence_id'] = data[1]
    df[feature] = data[feature]
    step_amount = i
    df['bins_fxd_step'] = fn.binning_fxd_step(data[feature], i)
    node_number = df['bins_fxd_step'].nunique()
    node_numbers.append(node_number)
    df['bins_fxd_bucket'] = fn.binning_fxd_bucket(data[feature],node_number)
    dfs.append(df)
    bucket_amount = max(df['bins_fxd_bucket'].value_counts())
    bucket_amounts.append(bucket_amount)
print(step_amounts)
print(node_numbers)
print(bucket_amounts)

[5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
[108, 100, 67, 51, 41, 34, 29, 26, 23, 21]
[3217, 3475, 5186, 6813, 8474, 10219, 11980, 13363, 15106, 16544]


In [18]:
matrices_step = []
matrices_bucket = []
bimatrices_step = []
bimatrices_bucket = []
for i in dfs:
    lift_pairwise_step, elem_map_step = fn.calculate_lift_pairwise(i, bins_column='bins_fxd_step', return_element_mapping=True)
    lift_pairwise_bucket, elem_map_bucket = fn.calculate_lift_pairwise(i, bins_column='bins_fxd_bucket', return_element_mapping=True)

    # Build adjacency matrices
    adj_matrix_step = fn.create_adjacency_matrix(lift_pairwise_step, elem_map_step)
    matrices_step.append(adj_matrix_step)
    adj_matrix_bucket = fn.create_adjacency_matrix(lift_pairwise_bucket, elem_map_bucket)
    matrices_bucket.append(adj_matrix_bucket)
    # Build binary matrices
    binary_matrix = adj_matrix_step.copy()
    binary_matrix[binary_matrix < 1] = 0
    binary_matrix[binary_matrix > 1] = 1
    bimatrices_step.append(binary_matrix)
    binary_matrix = adj_matrix_bucket.copy()
    binary_matrix[binary_matrix < 1] = 0
    binary_matrix[binary_matrix > 1] = 1
    bimatrices_bucket.append(binary_matrix)